# Introduction

In this section I will be using Folium to see if there is a relationship between the launch site and the landing success.

In [2]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

df = pd.read_csv('datasets/launch_data_falcon9_wiki.csv')

df.head()

,Flight_Number,Launch_Site,Payload,Payload_Mass,Orbit,Customer,Mission_Outcome,Booster_Version,Landing_Outcome,Latitude,Longitude,Datetime,Class
0,1,CCSFS SLC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,F9 v1.0 B0003.1,Failure (parachute),28.563197,-80.57682,2010-06-04 18:45:00,0
1,2,CCSFS SLC-40,Dragon,0.0,LEO,NASA (COTS) NRO,Success,F9 v1.0 B0004.1,Failure (parachute),28.563197,-80.57682,2010-12-08 15:43:00,0
2,3,CCSFS SLC-40,Dragon,525.0,LEO,NASA (COTS),Success,F9 v1.0 B0005.1,No attempt,28.563197,-80.57682,2012-05-22 07:44:00,0
3,4,CCSFS SLC-40,SpaceX CRS-1,4700.0,LEO,NASA (CRS),Success,F9 v1.0 B0006.1,No attempt,28.563197,-80.57682,2012-10-08 00:35:00,0
4,5,CCSFS SLC-40,SpaceX CRS-2,4877.0,LEO,NASA (CRS),Success,F9 v1.0 B0007.1,No attempt,28.563197,-80.57682,2013-03-01 15:10:00,0


## Task 1: Mark all launch sites on a map

In [6]:
# select relevant columns
spacex_df = df[['Launch_Site', 'Latitude', 'Longitude', 'Class']]

# create a dataframe of just the three launch sites and their locations
launch_sites_df = spacex_df.groupby('Launch_Site', as_index=False).first()
launch_sites_df = launch_sites_df[['Launch_Site', 'Latitude', 'Longitude']]
launch_sites_df['ShortName'] = ['CCSFS', 'CCSFS', 'KSC', 'VAFB']

launch_sites_df

,Launch_Site,Latitude,Longitude,ShortName
0,CCSFS LC-40,28.562302,-80.577356,CCSFS
1,CCSFS SLC-40,28.563197,-80.576820,CCSFS
2,KSC LC-39A,28.573255,-80.646895,KSC
3,VAFB SLC-4E,34.632834,-120.610745,VAFB


First, let's create a Folium Map object with the NASA Johnson Space Center in Houston, Texas as the first pinpoint:

In [7]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

In [8]:
circle = folium.Circle(nasa_coordinate, radius=1000, 
                       color='#d35400', 
                       fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(icon_size=(20,20),
                 icon_anchor=(0,0), 
                 html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC')
)

site_map.add_child(circle)
site_map.add_child(marker)

In [10]:
# add a circle and marker for each launch site
launch_map = folium.Map(location=nasa_coordinate, zoom_start=4)

for short, name, lat, long in zip(launch_sites_df['ShortName'], launch_sites_df['Launch_Site'], launch_sites_df['Latitude'], launch_sites_df['Longitude']):
    circle = folium.Circle([lat, long], radius=1000, 
                       color='#d35400', 
                       fill=True).add_child(folium.Popup(f'Launch from {name}'))

    marker = folium.map.Marker(
        [lat, long],
        icon=DivIcon(icon_size=(20,20),
                     icon_anchor=(0,0), 
                     html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % short)
    )

    launch_map.add_child(circle)
    launch_map.add_child(marker)
    print(f'Added {name} to map.')

Added CCSFS LC-40 to map.
Added CCSFS SLC-40 to map.
Added KSC LC-39A to map.
Added VAFB SLC-4E to map.


In [11]:
launch_map

From the map, we can see that all the launch sites are close to the coasts, with VAFB being on the west coast of the United States and CCSFS and KSC both on the east. The launch sites are also fairly close to the equator, or at least in locations of the United States that are closest to the equator rather than the northern areas of the country.

## Task 2: Mark the success/failed launches for each site on the map

Markers for each launch record will be added to the map to see which sites have high success rates. A green marker will be used for a successful landing (Class = 1) and a red one for an unsuccessful landing (Class = 0).

Since a launch only happens in one of the three launch sites, there will be many launch records with the same coordinate. `MarkerClusters` will be useful here.

In [13]:
marker_cluster = MarkerCluster()

# create a column in the dataframe with the color for each launch
spacex_df['Color'] = spacex_df['Class'].map(lambda x: 'green' if x else 'red')

spacex_df.head()

/var/folders/0r/wbzfff2x7xn34__m82hhzyj40000gn/T/ipykernel_24565/2509329330.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spacex_df['Color'] = spacex_df['Class'].map(lambda x: 'green' if x else 'red')


,Launch_Site,Latitude,Longitude,Class,Color
0,CCSFS SLC-40,28.563197,-80.57682,0,red
1,CCSFS SLC-40,28.563197,-80.57682,0,red
2,CCSFS SLC-40,28.563197,-80.57682,0,red
3,CCSFS SLC-40,28.563197,-80.57682,0,red
4,CCSFS SLC-40,28.563197,-80.57682,0,red


In [14]:
for i, launch in spacex_df.iterrows():
    marker = folium.map.Marker(
        [launch.Latitude, launch.Longitude],
        icon=folium.Icon(color='white', icon_color=launch.Color)
    )
    marker_cluster.add_child(marker)
launch_map.add_child(marker_cluster)
launch_map